### Twython works fast to up to 3200 tweets
This notebook goes through a user's timeline and collects up to 3200 of the most recent tweets.

In [10]:
import json
import pandas as pd
from twython import Twython, TwythonError

In [11]:
with open("..\..\..\data\interim\credentials.json") as infile:
    credentials = json.load(infile)

In [12]:
twitter_client = Twython(credentials["consumer_key"],
                         credentials["consumer_secret"],
                         credentials["access_token"],
                         credentials["access_token_secret"])

In [13]:
def load_ext_user_timeline(twitter_client, screen_name):
    tweets = []
    try:
        user_timeline = twitter_client.get_user_timeline(screen_name=screen_name, tweet_mode='extended')
    except TwythonError as e:
        print(e)
    for tweet in user_timeline:
        # Add whatever you want from the tweet, here we just add the text
        tweets.append(tweet)
    # Count could be less than 200, see:
    while len(user_timeline) != 0:
        try:
            user_timeline = twitter_client.get_user_timeline(
                screen_name=screen_name,count=200,max_id=user_timeline[len(user_timeline)-1]['id']-1, tweet_mode='extended')
        except TwythonError as e:
            print(e)
        for tweet in user_timeline:
            
            tweets.append(tweet)
    
    print(len(tweets))
    return tweets

In [18]:
twitter_account = 'ManUtd'
mu_tweets_list = load_ext_user_timeline(twitter_client, twitter_account)

3251


In [19]:
mu_df = pd.DataFrame(mu_tweets_list)
print(mu_df.shape)
mu_df.tail()

(3251, 31)


,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status_permalink,quoted_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,retweeted_status
3246,Wed May 04 11:27:18 +0000 2022,1521813715418591233,1521813715418591233,ℹ️ We are revising our season-ticket policies ...,False,"[0, 124]","{'hashtags': [{'text': 'MUFC', 'indices': [119...","<a href=""https://twitter.com"" rel=""nofollow"">T...",NaN,None,...,NaN,NaN,162,2427,False,False,NaN,en,NaN,NaN
3247,Wed May 04 11:02:00 +0000 2022,1521807348360105984,1521807348360105984,No less than 1.75 million reasons to give back...,False,"[0, 274]","{'hashtags': [{'text': 'MUFC', 'indices': [269...","<a href=""https://twitter.com"" rel=""nofollow"">T...",NaN,None,...,NaN,NaN,204,2329,False,False,NaN,en,NaN,NaN
3248,Wed May 04 09:15:58 +0000 2022,1521780662977630210,1521780662977630210,A 5️⃣0️⃣th goal in red for @B_Fernandes8 💥👏\n\...,False,"[0, 60]","{'hashtags': [{'text': 'MUFC', 'indices': [45,...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,301,4646,False,False,False,en,"{'media': [{'id': 1521780614600564741, 'id_str...",NaN
3249,Wed May 04 05:58:00 +0000 2022,1521730843739262977,1521730843739262977,The Matic family 👆\n\n♥ @NemanjaMatic\n\n#MUFC...,False,"[0, 42]","{'hashtags': [{'text': 'MUFC', 'indices': [37,...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,NaN,NaN,1217,29609,False,False,False,en,"{'media': [{'id': 1521486208479772673, 'id_str...",NaN
3250,Wed May 04 03:12:00 +0000 2022,1521689068618207232,1521689068618207232,There were some difficult decisions to be made...,False,"[0, 64]","{'hashtags': [{'text': 'MUFC', 'indices': [53,...","<a href=""https://twitter.com"" rel=""nofollow"">T...",NaN,None,...,NaN,NaN,139,3077,False,False,NaN,en,NaN,NaN


In [15]:
mu_df.to_csv("../../../data/raw/mu_6m.csv")

Funtion to summarize each user's key information

In [16]:
def summarize(tweet, extra_fields = None):
    new_tweet = {}
    for field, value in tweet.items():
        if field in ["text", "id_str", "screen_name", "retweet_count", "favorite_count", "in_reply_to_status_id_str", "in_reply_to_screen_name", "in_reply_to_user_id_str"] and value is not None:
            new_tweet[field] = value
        elif extra_fields and field in extra_fields:
            new_tweet[field] = value
        elif field in ["retweeted_status", "quoted_status", "user"]:
            new_tweet[field] = summarize(value)
    return new_tweet

In [18]:
summarize(mu_tweets_list[5])

{'id_str': '1588311774373482497',
 'user': {'id_str': '156247074', 'screen_name': 'WestHam'},
 'retweet_count': 155,
 'favorite_count': 2173}

In [20]:
info_list = []
for i in range(len(mu_tweets_list)):
    info_list.append(summarize(mu_tweets_list[i]))

In [22]:
column_list = []
for item in info_list:
    for k in item.keys():
        column_list.append(k)

In [26]:
post_info = pd.DataFrame(info_list, columns=column_list)

In [27]:
post_info

,id_str,user,retweet_count,favorite_count,id_str,user,retweet_count,favorite_count,id_str,user,...,retweet_count,favorite_count,id_str,user,retweet_count,favorite_count,id_str,user,retweet_count,favorite_count
0,1588510024930070530,"{'id_str': '156247074', 'screen_name': 'WestHam'}",29,370,1588510024930070530,"{'id_str': '156247074', 'screen_name': 'WestHam'}",29,370,1588510024930070530,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,29,370,1588510024930070530,"{'id_str': '156247074', 'screen_name': 'WestHam'}",29,370,1588510024930070530,"{'id_str': '156247074', 'screen_name': 'WestHam'}",29,370
1,1588495897226145792,"{'id_str': '156247074', 'screen_name': 'WestHam'}",88,1022,1588495897226145792,"{'id_str': '156247074', 'screen_name': 'WestHam'}",88,1022,1588495897226145792,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,88,1022,1588495897226145792,"{'id_str': '156247074', 'screen_name': 'WestHam'}",88,1022,1588495897226145792,"{'id_str': '156247074', 'screen_name': 'WestHam'}",88,1022
2,1588482437482532866,"{'id_str': '156247074', 'screen_name': 'WestHam'}",39,967,1588482437482532866,"{'id_str': '156247074', 'screen_name': 'WestHam'}",39,967,1588482437482532866,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,39,967,1588482437482532866,"{'id_str': '156247074', 'screen_name': 'WestHam'}",39,967,1588482437482532866,"{'id_str': '156247074', 'screen_name': 'WestHam'}",39,967
3,1588472535443591168,"{'id_str': '156247074', 'screen_name': 'WestHam'}",33,723,1588472535443591168,"{'id_str': '156247074', 'screen_name': 'WestHam'}",33,723,1588472535443591168,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,33,723,1588472535443591168,"{'id_str': '156247074', 'screen_name': 'WestHam'}",33,723,1588472535443591168,"{'id_str': '156247074', 'screen_name': 'WestHam'}",33,723
4,1588463833727000576,"{'id_str': '156247074', 'screen_name': 'WestHam'}",41,900,1588463833727000576,"{'id_str': '156247074', 'screen_name': 'WestHam'}",41,900,1588463833727000576,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,41,900,1588463833727000576,"{'id_str': '156247074', 'screen_name': 'WestHam'}",41,900,1588463833727000576,"{'id_str': '156247074', 'screen_name': 'WestHam'}",41,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,1495022948603863054,"{'id_str': '156247074', 'screen_name': 'WestHam'}",59,1602,1495022948603863054,"{'id_str': '156247074', 'screen_name': 'WestHam'}",59,1602,1495022948603863054,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,59,1602,1495022948603863054,"{'id_str': '156247074', 'screen_name': 'WestHam'}",59,1602,1495022948603863054,"{'id_str': '156247074', 'screen_name': 'WestHam'}",59,1602
3245,1495021691231256579,"{'id_str': '156247074', 'screen_name': 'WestHam'}",53,715,1495021691231256579,"{'id_str': '156247074', 'screen_name': 'WestHam'}",53,715,1495021691231256579,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,53,715,1495021691231256579,"{'id_str': '156247074', 'screen_name': 'WestHam'}",53,715,1495021691231256579,"{'id_str': '156247074', 'screen_name': 'WestHam'}",53,715
3246,1495021029684658181,"{'id_str': '156247074', 'screen_name': 'WestHam'}",158,1477,1495021029684658181,"{'id_str': '156247074', 'screen_name': 'WestHam'}",158,1477,1495021029684658181,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,158,1477,1495021029684658181,"{'id_str': '156247074', 'screen_name': 'WestHam'}",158,1477,1495021029684658181,"{'id_str': '156247074', 'screen_name': 'WestHam'}",158,1477
3247,1495018396848697344,"{'id_str': '156247074', 'screen_name': 'WestHam'}",10,271,1495018396848697344,"{'id_str': '156247074', 'screen_name': 'WestHam'}",10,271,1495018396848697344,"{'id_str': '156247074', 'screen_name': 'WestHam'}",...,10,271,1495018396848697344,"{'id_str': '156247074', 'screen_name': 'WestHam'}",10,271,1495018396848697344,"{'id_str': '156247074', 'screen_name': 'WestHam'}",10,271
